In [ ]:
import requests
from datetime import datetime, timedelta

# ============================================================================
# CONFIGURATION - Import secrets from secure file
# ============================================================================
from secrets_config import GITHUB_TOKEN, GITHUB_REPO

# ============================================================================
# DELETION CONFIGURATION - Modify these as needed
# ============================================================================

# Default number of posts to delete when running quick deletion
DEFAULT_NUM_TO_DELETE = 3

# Date range for bulk deletion (set to None to disable)
START_DATE = "2025-05-27"  # Format: "YYYY-MM-DD" or None
END_DATE = "2025-05-27"    # Format: "YYYY-MM-DD" or None

# Quick settings for common operations
DELETE_TODAY_POSTS = False      # Set to True to auto-delete today's posts
DELETE_LAST_N_POSTS = 0         # Set to number > 0 to auto-delete recent posts
DELETE_DATE_RANGE = False       # Set to True to auto-delete posts in date range

# Safety settings
REQUIRE_CONFIRMATION = True     # Ask before deleting
SHOW_PREVIEW = True            # Show what will be deleted before confirming
MAX_AUTO_DELETE = 10           # Maximum posts to auto-delete (safety limit)
MAX_DATE_RANGE_DAYS = 30       # Maximum days in date range (safety limit)

# ============================================================================
# DELETION FUNCTIONS
# ============================================================================

def list_blog_posts():
    """List all blog posts in the _posts directory"""
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        posts = response.json()
        blog_posts = []
        
        for post in posts:
            if post['name'].endswith('.md'):
                blog_posts.append({
                    'name': post['name'],
                    'path': post['path'],
                    'sha': post['sha'],
                    'download_url': post['download_url']
                })
        
        # Sort by name (which includes date) - newest first
        blog_posts.sort(key=lambda x: x['name'], reverse=True)
        return blog_posts
        
    except requests.RequestException as e:
        print(f"Error fetching blog posts: {e}")
        return []

def delete_blog_post(post_info):
    """Delete a single blog post"""
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{post_info['path']}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    data = {
        "message": f"Delete blog post: {post_info['name']}",
        "sha": post_info['sha']
    }
    
    try:
        response = requests.delete(url, headers=headers, json=data)
        response.raise_for_status()
        print(f"✅ Deleted: {post_info['name']}")
        return True
        
    except requests.RequestException as e:
        print(f"❌ Failed to delete {post_info['name']}: {e}")
        return False

def delete_latest_blogs(num_to_delete=None, confirm=None):
    """Delete the most recent blog posts"""
    if num_to_delete is None:
        num_to_delete = DEFAULT_NUM_TO_DELETE
    if confirm is None:
        confirm = REQUIRE_CONFIRMATION
        
    print(f"Fetching list of blog posts...")
    posts = list_blog_posts()
    
    if not posts:
        print("No blog posts found.")
        return
    
    if num_to_delete > len(posts):
        print(f"You asked to delete {num_to_delete} posts, but only {len(posts)} exist.")
        num_to_delete = len(posts)
    
    posts_to_delete = posts[:num_to_delete]
    
    if SHOW_PREVIEW:
        print(f"\nFound {len(posts)} total blog posts.")
        print(f"Will delete the {num_to_delete} most recent posts:")
        print("-" * 50)
        
        for i, post in enumerate(posts_to_delete, 1):
            print(f"{i}. {post['name']}")
    
    if confirm:
        print("-" * 50)
        confirmation = input(f"\nAre you sure you want to delete these {num_to_delete} posts? (yes/no): ")
        if confirmation.lower() not in ['yes', 'y']:
            print("Deletion cancelled.")
            return
    
    print(f"\nDeleting {num_to_delete} blog posts...")
    deleted_count = 0
    
    for post in posts_to_delete:
        if delete_blog_post(post):
            deleted_count += 1
    
    print(f"\n✅ Successfully deleted {deleted_count} out of {num_to_delete} blog posts.")

def delete_blogs_by_date_range(start_date=None, end_date=None):
    """Delete all blog posts within a date range"""
    if start_date is None:
        start_date = START_DATE
    if end_date is None:
        end_date = END_DATE
        
    if not start_date or not end_date:
        print("Error: Both start_date and end_date must be specified")
        return
        
    print(f"Fetching blog posts between {start_date} and {end_date}...")
    posts = list_blog_posts()
    
    posts_to_delete = []
    for post in posts:
        # Extract date from filename (format: YYYY-MM-DD-title.md)
        post_date = post['name'][:10]  # First 10 characters should be YYYY-MM-DD
        if start_date <= post_date <= end_date:
            posts_to_delete.append(post)
    
    if not posts_to_delete:
        print(f"No blog posts found between {start_date} and {end_date}")
        return
    
    if SHOW_PREVIEW:
        print(f"Found {len(posts_to_delete)} blog posts in date range:")
        for post in posts_to_delete:
            print(f"  - {post['name']}")
    
    if REQUIRE_CONFIRMATION:
        confirmation = input(f"\nDelete all {len(posts_to_delete)} posts from {start_date} to {end_date}? (yes/no): ")
        if confirmation.lower() not in ['yes', 'y']:
            print("Deletion cancelled.")
            return
    
    deleted_count = 0
    for post in posts_to_delete:
        if delete_blog_post(post):
            deleted_count += 1
    
    print(f"\n✅ Successfully deleted {deleted_count} out of {len(posts_to_delete)} blog posts.")

def delete_blogs_by_single_date(date_string):
    """Delete all blog posts from a specific date"""
    print(f"Fetching blog posts from {date_string}...")
    posts = list_blog_posts()
    
    posts_to_delete = [post for post in posts if post['name'].startswith(date_string)]
    
    if not posts_to_delete:
        print(f"No blog posts found for date {date_string}")
        return
    
    if SHOW_PREVIEW:
        print(f"Found {len(posts_to_delete)} blog posts from {date_string}:")
        for post in posts_to_delete:
            print(f"  - {post['name']}")
    
    if REQUIRE_CONFIRMATION:
        confirmation = input(f"\nDelete all {len(posts_to_delete)} posts from {date_string}? (yes/no): ")
        if confirmation.lower() not in ['yes', 'y']:
            print("Deletion cancelled.")
            return
    
    deleted_count = 0
    for post in posts_to_delete:
        if delete_blog_post(post):
            deleted_count += 1
    
    print(f"\n✅ Successfully deleted {deleted_count} out of {len(posts_to_delete)} blog posts.")

def show_all_posts():
    """Display all blog posts with numbers for easy reference"""
    posts = list_blog_posts()
    
    if not posts:
        print("No blog posts found.")
        return
    
    print(f"Found {len(posts)} blog posts:")
    print("-" * 70)
    
    for i, post in enumerate(posts, 1):
        print(f"{i:2d}. {post['name']}")
    
    return posts

def auto_delete():
    """Run automatic deletion based on configuration"""
    
    # Safety check: Don't auto-delete if confirmation is disabled and limits are high
    if not REQUIRE_CONFIRMATION:
        total_potential_deletes = DELETE_LAST_N_POSTS
        if total_potential_deletes > MAX_AUTO_DELETE:
            print(f"🚨 SAFETY CHECK: Refusing to auto-delete {total_potential_deletes} posts without confirmation!")
            print(f"   Maximum allowed: {MAX_AUTO_DELETE}")
            print("   Either enable REQUIRE_CONFIRMATION or reduce the number.")
            return
    
    if DELETE_TODAY_POSTS:
        print("🤖 Auto-deleting today's posts...")
        delete_blogs_by_single_date(datetime.now().strftime("%Y-%m-%d"))
    
    if DELETE_LAST_N_POSTS > 0:
        if DELETE_LAST_N_POSTS > MAX_AUTO_DELETE and not REQUIRE_CONFIRMATION:
            print(f"🚨 SAFETY CHECK: Refusing to auto-delete {DELETE_LAST_N_POSTS} posts without confirmation!")
            print(f"   Maximum allowed: {MAX_AUTO_DELETE}")
            return
        print(f"🤖 Auto-deleting last {DELETE_LAST_N_POSTS} posts...")
        delete_latest_blogs(DELETE_LAST_N_POSTS)
    
    if DELETE_DATE_RANGE and START_DATE and END_DATE:
        # Safety check: validate date range
        try:
            start_dt = datetime.strptime(START_DATE, "%Y-%m-%d")
            end_dt = datetime.strptime(END_DATE, "%Y-%m-%d")
            days_diff = (end_dt - start_dt).days
            
            if days_diff > MAX_DATE_RANGE_DAYS:
                print(f"🚨 SAFETY CHECK: Date range too large ({days_diff} days)")
                print(f"   Maximum allowed: {MAX_DATE_RANGE_DAYS} days")
                print("   Please use a smaller date range.")
                return
                
            if days_diff < 0:
                print(f"🚨 SAFETY CHECK: Start date is after end date!")
                return
                
        except ValueError:
            print(f"🚨 SAFETY CHECK: Invalid date format in START_DATE or END_DATE")
            return
            
        print(f"🤖 Auto-deleting posts from {START_DATE} to {END_DATE}...")
        delete_blogs_by_date_range()

# Usage examples
if __name__ == "__main__":
    print("BLOG POST DELETION TOOL")
    print("=" * 50)
    
    # Check if auto-deletion is configured
    if DELETE_TODAY_POSTS or DELETE_LAST_N_POSTS > 0 or DELETE_DATE_RANGE:
        print("Auto-deletion is configured. Running automatic deletion...")
        auto_delete()
    else:
        # Interactive mode
        while True:
            print("\nOptions:")
            print("1. Show all blog posts")
            print("2. Delete latest N posts")
            print("3. Delete posts by single date")
            print("4. Delete posts by date range")
            print(f"5. Delete last {DEFAULT_NUM_TO_DELETE} posts (quick)")
            print("6. Delete today's posts (quick)")
            print("7. Exit")
            
            choice = input("\nEnter your choice (1-7): ").strip()
            
            if choice == "1":
                show_all_posts()
                
            elif choice == "2":
                try:
                    num = int(input("How many recent posts to delete? "))
                    delete_latest_blogs(num)
                except ValueError:
                    print("Please enter a valid number.")
                    
            elif choice == "3":
                date_str = input("Enter date (YYYY-MM-DD): ").strip()
                delete_blogs_by_single_date(date_str)
                
            elif choice == "4":
                start = input("Enter start date (YYYY-MM-DD): ").strip()
                end = input("Enter end date (YYYY-MM-DD): ").strip()
                delete_blogs_by_date_range(start, end)
                
            elif choice == "5":
                delete_latest_blogs()
                
            elif choice == "6":
                today = datetime.now().strftime("%Y-%m-%d")
                delete_blogs_by_single_date(today)
                
            elif choice == "7":
                print("Goodbye!")
                break
                
            else:
                print("Invalid choice. Please try again.")